# Differential Privacy for PRETSA

In [1]:
!git clone https://github.com/cmalvr/PRETSA

Cloning into 'PRETSA'...
remote: Enumerating objects: 211, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 211 (delta 49), reused 43 (delta 19), pack-reused 115 (from 1)
Receiving objects: 100% (211/211), 11.45 MiB | 8.47 MiB/s, done.
Resolving deltas: 100% (114/114), done.


In [2]:
!pip install -r /content/PRETSA/requirements.txt

  Preparing metadata (setup.py) ... done
  Created wheel for anytree: filename=anytree-2.4.3-py2.py3-none-any.whl size=26707 sha256=46c2fd8082a490951c57bc782ec762af0cb4571101a0ecadc551a8d19ddcb6a8
  Stored in directory: /root/.cache/pip/wheels/f3/d3/8f/9eea88a7fe27ca18e7ebfc23e1012514c8b13a4552aa43be12
Successfully built anytree


In [3]:
!rm -r "/content/PRETSA/baselogs"

rm: cannot remove '/content/PRETSA/baselogs': No such file or directory


In [4]:
from pathlib import Path
import zipfile

# Define paths
zip_path = Path("/content/PRETSA/baselogs.zip")
extract_dir = Path("/content/PRETSA")

# Extract zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Search for CSV files
csv_files = list(extract_dir.rglob("*.csv"))

if csv_files:
    print("Found CSV files:", [f.name for f in csv_files])
else:
    print("No CSV files found.")

Found CSV files: ['bpic2013_dataset.csv', 'Sepsis_dataset.csv', 'traffic_fines_dataset.csv', 'HospitalBilling_dataset.csv', 'CoSeLoG_dataset.csv']


In [5]:
!python /content/PRETSA/add_annotation_duration.py Sepsis_dataset /content/PRETSA/baselogs/Sepsis_dataset.csv

/content/PRETSA/baselogs/Sepsis_dataset_duration.csv
